In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

In [48]:
data = pd.read_csv('train.csv')
data = data.dropna()

data["date"] = pd.to_datetime(data["date"])
data["year"] = data["date"].dt.year
data["month"] = data["date"].dt.month
data["day_of_week"] = data["date"].dt.dayofweek
data["is_weekend"] = data["day_of_week"].isin([5, 6]).astype(int)
data["quarter"] = data["date"].dt.quarter

# 3. Uzupełnienie brakujących wartości
# Wypełniamy medianą grupując po kraju, sklepie i produkcie
data["num_sold"] = data.groupby(["country", "store", "product"])["num_sold"].transform(lambda x: x.fillna(x.median()))

# 4. Dodanie lag_features (opóźnione wartości num_sold)
data = data.sort_values(by=["country", "store", "product", "date"])
data["lag_1"] = data.groupby(["country", "store", "product"])["num_sold"].shift(1)
data["lag_7"] = data.groupby(["country", "store", "product"])["num_sold"].shift(7)
data["lag_30"] = data.groupby(["country", "store", "product"])["num_sold"].shift(30)
data.fillna(0, inplace=True)  # Wypełniamy brakujące wartości zerami dla lagów

# 5. Kodowanie zmiennych kategorycznych
le_country = LabelEncoder()
data["country_encoded"] = le_country.fit_transform(data["country"])

le_store = LabelEncoder()
data["store_encoded"] = le_store.fit_transform(data["store"])

le_product = LabelEncoder()
data["product_encoded"] = le_product.fit_transform(data["product"])

# 6. Normalizacja cech
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(data[["year", "month", "day_of_week", "is_weekend", "quarter", "lag_1", "lag_7", "lag_30"]])
scaled_features_df = pd.DataFrame(scaled_features, columns=["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled", "lag_1_scaled", "lag_7_scaled", "lag_30_scaled"])
data = pd.concat([data.reset_index(drop=True), scaled_features_df], axis=1)

# 7. Przygotowanie zbioru treningowego i testowego
train_data = data[data["year"] < 2016]
test_data = data[data["year"] == 2016]

X_train = train_data[["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled", "lag_1_scaled", "lag_7_scaled", "lag_30_scaled", "country_encoded", "store_encoded", "product_encoded"]]
y_train = train_data["num_sold"]

X_test = test_data[["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled", "lag_1_scaled", "lag_7_scaled", "lag_30_scaled", "country_encoded", "store_encoded", "product_encoded"]]
y_test = test_data["num_sold"]

# 8. Trening modelu XGBoost
model = XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train, y_train)

# 9. Predykcja i ocena modelu
predictions = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, predictions)
print(f"MAPE: {mape:.3f}")




MAPE: 0.061


In [46]:
data

,id,date,country,store,product,num_sold,year,month,day_of_week,is_weekend,...,store_encoded,product_encoded,year_scaled,month,day_of_week,is_weekend,quarter,lag_1,lag_7,lag_30
0,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0,2010,1,4,0,...,0,1,0.0,0.0,0.666667,0.0,0.0,0.000000,0.000000,0.000000
1,91,2010-01-02,Canada,Discount Stickers,Kaggle,881.0,2010,1,5,1,...,0,1,0.0,0.0,0.833333,1.0,0.0,0.163832,0.000000,0.000000
2,181,2010-01-03,Canada,Discount Stickers,Kaggle,1003.0,2010,1,6,1,...,0,1,0.0,0.0,1.000000,1.0,0.0,0.148341,0.000000,0.000000
3,271,2010-01-04,Canada,Discount Stickers,Kaggle,744.0,2010,1,0,0,...,0,1,0.0,0.0,0.000000,0.0,0.0,0.168884,0.000000,0.000000
4,361,2010-01-05,Canada,Discount Stickers,Kaggle,707.0,2010,1,1,0,...,0,1,0.0,0.0,0.166667,0.0,0.0,0.125274,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221254,229764,2016-12-27,Singapore,Stickers for Less,Kerneler Dark Mode,1016.0,2016,12,1,0,...,2,4,1.0,1.0,0.166667,0.0,1.0,0.164674,0.145142,0.158949
221255,229854,2016-12-28,Singapore,Stickers for Less,Kerneler Dark Mode,1062.0,2016,12,2,0,...,2,4,1.0,1.0,0.333333,0.0,1.0,0.171073,0.139249,0.114666
221256,229944,2016-12-29,Singapore,Stickers for Less,Kerneler Dark Mode,1178.0,2016,12,3,0,...,2,4,1.0,1.0,0.500000,0.0,1.0,0.178818,0.145311,0.131167
221257,230034,2016-12-30,Singapore,Stickers for Less,Kerneler Dark Mode,1357.0,2016,12,4,0,...,2,4,1.0,1.0,0.666667,0.0,1.0,0.198350,0.150025,0.133356


In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

In [65]:
data = pd.read_csv('train.csv')
data = data.dropna()


data["date"] = pd.to_datetime(data["date"])
data["year"] = data["date"].dt.year
data["month"] = data["date"].dt.month
data["day_of_week"] = data["date"].dt.dayofweek
data["is_weekend"] = data["day_of_week"].isin([5, 6]).astype(int)
data["quarter"] = data["date"].dt.quarter

# 3. Uzupełnienie brakujących wartości
# Wypełniamy medianą grupując po kraju, sklepie i produkcie
data["num_sold"] = data.groupby(["country", "store", "product"])["num_sold"].transform(lambda x: x.fillna(x.median()))

# 4. Dodanie lag_features (opóźnione wartości num_sold)
data = data.sort_values(by=["country", "store", "product", "date"])
data["lag_1"] = data.groupby(["country", "store", "product"])["num_sold"].shift(1)
data["lag_7"] = data.groupby(["country", "store", "product"])["num_sold"].shift(7)
data["lag_30"] = data.groupby(["country", "store", "product"])["num_sold"].shift(30)
data.fillna(0, inplace=True)  # Wypełniamy brakujące wartości zerami dla lagów

# 5. Kodowanie zmiennych kategorycznych
le_country = LabelEncoder()
data["country_encoded"] = le_country.fit_transform(data["country"])

le_store = LabelEncoder()
data["store_encoded"] = le_store.fit_transform(data["store"])

le_product = LabelEncoder()
data["product_encoded"] = le_product.fit_transform(data["product"])

# 6. Normalizacja cech
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(data[["year", "month", "day_of_week", "is_weekend", "quarter", "lag_1", "lag_7", "lag_30"]])
scaled_features_df = pd.DataFrame(scaled_features, columns=["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled", "lag_1_scaled", "lag_7_scaled", "lag_30_scaled"])
data = pd.concat([data.reset_index(drop=True), scaled_features_df], axis=1)

# 7. Przygotowanie zbioru treningowego i testowego
train_data = data[data["year"] < 2016]
test_data = data[data["year"] == 2016]

X_train = train_data[["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled", "lag_1_scaled", "lag_7_scaled", "lag_30_scaled", "country_encoded", "store_encoded", "product_encoded"]]
y_train = train_data["num_sold"]

X_test = test_data[["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled", "lag_1_scaled", "lag_7_scaled", "lag_30_scaled", "country_encoded", "store_encoded", "product_encoded"]]
y_test = test_data["num_sold"]

# 8. Definicja modeli
xgb_model = XGBRegressor(objective="reg:squarederror", random_state=42)
lgbm_model = LGBMRegressor(random_state=42, verbose=-1)
catboost_model = CatBoostRegressor(silent=True, random_state=42)

# 9. Ensamblowanie modeli
ensemble_model = VotingRegressor([
    ("xgb", xgb_model),
    ("lgbm", lgbm_model),
    ("catboost", catboost_model)
])

# 10. Optymalizacja parametrów
param_grid = {
    "xgb__n_estimators": [50, 100],
    "xgb__learning_rate": [0.05, 0.1],
    "xgb__max_depth": [4, 6],
    "lgbm__n_estimators": [50, 100],
    "lgbm__learning_rate": [0.05, 0.1],
    "lgbm__max_depth": [4, 6],
    "catboost__depth": [4, 6],
    "catboost__iterations": [50, 100],
    "catboost__learning_rate": [0.05, 0.1]
}

search = GridSearchCV(estimator=ensemble_model, param_grid=param_grid, cv=3, scoring="neg_mean_absolute_percentage_error")
search.fit(X_train, y_train)

# 11. Predykcja i ocena modelu
best_model = search.best_estimator_
predictions = best_model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, predictions)
print(f"MAPE (ensemble model): {mape:.3f}")

#Ocena poszczególnych modeli
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_mape = mean_absolute_percentage_error(y_test, xgb_predictions)
print(f"MAPE (XGBoost): {xgb_mape:.3f}")

lgbm_model.fit(X_train, y_train)
lgbm_predictions = lgbm_model.predict(X_test)
lgbm_mape = mean_absolute_percentage_error(y_test, lgbm_predictions)
print(f"MAPE (LightGBM): {lgbm_mape:.3f}")

catboost_model.fit(X_train, y_train)
catboost_predictions = catboost_model.predict(X_test)
catboost_mape = mean_absolute_percentage_error(y_test, catboost_predictions)
print(f"MAPE (CatBoost): {catboost_mape:.3f}")

MAPE (ensemble model): 0.898
MAPE (XGBoost): 0.063
MAPE (LightGBM): 0.069
MAPE (CatBoost): 0.078


In [54]:
mape

0.06142343264953543

In [78]:
data = pd.read_csv('train.csv')
data = data.dropna()

data["date"] = pd.to_datetime(data["date"])
data["year"] = data["date"].dt.year
data["month"] = data["date"].dt.month
data["day_of_week"] = data["date"].dt.dayofweek
data["is_weekend"] = data["day_of_week"].isin([5, 6]).astype(int)
data["quarter"] = data["date"].dt.quarter

# 3. Uzupełnienie brakujących wartości
# Wypełniamy medianą grupując po kraju, sklepie i produkcie
data["num_sold"] = data.groupby(["country", "store", "product"])["num_sold"].transform(lambda x: x.fillna(x.median()))

# 4. Dodanie lag_features (opóźnione wartości num_sold)
data = data.sort_values(by=["country", "store", "product", "date"])
# data["lag_1"] = data.groupby(["country", "store", "product"])["num_sold"].shift(1)
# data["lag_7"] = data.groupby(["country", "store", "product"])["num_sold"].shift(7)
# data["lag_30"] = data.groupby(["country", "store", "product"])["num_sold"].shift(30)
data.fillna(0, inplace=True)  # Wypełniamy brakujące wartości zerami dla lagów

# 5. Kodowanie zmiennych kategorycznych
le_country = LabelEncoder()
data["country_encoded"] = le_country.fit_transform(data["country"])

le_store = LabelEncoder()
data["store_encoded"] = le_store.fit_transform(data["store"])

le_product = LabelEncoder()
data["product_encoded"] = le_product.fit_transform(data["product"])

# 6. Normalizacja cech
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(data[["year", "month", "day_of_week", "is_weekend", "quarter"]])
scaled_features_df = pd.DataFrame(scaled_features, columns=["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled"])
data = pd.concat([data.reset_index(drop=True), scaled_features_df], axis=1)

# 7. Przygotowanie zbioru treningowego i testowego
train_data = data #[data["year"] < 2017]
# test_data = data[data["year"] == 2016]

X_train = train_data[["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled", "country_encoded", "store_encoded", "product_encoded"]]
y_train = train_data["num_sold"]

# X_test = test_data[["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled", "lag_1_scaled", "lag_7_scaled", "lag_30_scaled", "country_encoded", "store_encoded", "product_encoded"]]
# y_test = test_data["num_sold"]

# 8. Trening modelu XGBoost
model = XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train, y_train)

# 9. Predykcja i ocena modelu
# predictions = model.predict(X_test)
# mape = mean_absolute_percentage_error(y_test, predictions)
# print(f"MAPE: {mape:.3f}")




XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [97]:
# Załaduj nowe dane testowe
new_data = pd.read_csv('test.csv')  # Twoje dane testowe z lat 2017-2019
new_data["date"] = pd.to_datetime(new_data["date"])

# Wyciągnij cechy daty
new_data["year"] = new_data["date"].dt.year
new_data["month"] = new_data["date"].dt.month
new_data["day_of_week"] = new_data["date"].dt.dayofweek
new_data["is_weekend"] = new_data["day_of_week"].isin([5, 6]).astype(int)
new_data["quarter"] = new_data["date"].dt.quarter
new_data["num_sold"] = None
# Połącz dane historyczne i nowe dane testowe
combined_data = pd.concat([data, new_data], ignore_index=True)

# Sortuj dane przed obliczaniem lagów
combined_data = combined_data.sort_values(by=["country", "store", "product", "date"])

# Dodaj lag features
# combined_data["lag_1"] = combined_data.groupby(["country", "store", "product"])["num_sold"].shift(1)
# combined_data["lag_7"] = combined_data.groupby(["country", "store", "product"])["num_sold"].shift(7)
# combined_data["lag_30"] = combined_data.groupby(["country", "store", "product"])["num_sold"].shift(30)

# Wypełnij brakujące lag wartości zerami (tylko dla nowych danych testowych)
# combined_data.loc[combined_data["num_sold"].isna(), ["lag_1", "lag_7", "lag_30"]] = 0

# Zakoduj zmienne kategoryczne
combined_data["country_encoded"] = le_country.transform(combined_data["country"])
combined_data["store_encoded"] = le_store.transform(combined_data["store"])
combined_data["product_encoded"] = le_product.transform(combined_data["product"])

# Skalowanie cech
scaled_features = scaler.transform(combined_data[["year", "month", "day_of_week", "is_weekend", "quarter"]])
scaled_features_df = pd.DataFrame(scaled_features, columns=["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled"])

# Dodaj znormalizowane cechy do danych
combined_data = pd.concat([combined_data.reset_index(drop=True), scaled_features_df], axis=1)

# Wybierz tylko nowe dane testowe
new_test_data = combined_data[combined_data["num_sold"].isna()]
new_test_data = new_test_data.drop_duplicates(subset=["id"])
# Przygotuj cechy do predykcji
X_new_test = new_test_data[["year_scaled", "month_scaled", "day_of_week_scaled", "is_weekend_scaled", "quarter_scaled", "country_encoded", "store_encoded", "product_encoded"]]
X_new_test = X_new_test.loc[:, ~X_new_test.columns.duplicated()]

# Dokonaj predykcji
new_test_data["num_sold"] = model.predict(X_new_test)

# Wynik
predicted_values = new_test_data[["id", "date", "country", "store", "product", "num_sold"]]
predicted_values.to_csv("predicted_sales.csv", index=False)


In [99]:
df = pd.read_csv('predicted_sales.csv')
df = df[['id','num_sold']]
df.to_csv("first_test.csv", index=False)

In [101]:
df

,id,num_sold
0,230130,153.63998
1,230220,114.40173
2,230310,114.40173
3,230400,114.40173
4,230490,114.40173
...,...,...
98545,328314,922.20810
98546,328404,964.58940
98547,328494,1053.35470
98548,328584,869.29846
